In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
from tkinter import simpledialog
from tkinter import filedialog
from geopy.geocoders import Nominatim, ArcGIS,MapBox
import geopandas as gpd
from functools import  partial

%matplotlib inline

In [3]:
dirDeArchivo = filedialog.askopenfilename(title="Abrir Archivo excel")
TipoArchivo = simpledialog.askstring(title="Que tipo de Archvo es ? Medico o Institución",prompt="ingresa la palabra Medico o Institucion",initialvalue="MEDICO").upper()
print(TipoArchivo)

verExcelOriginal= pd.read_excel(dirDeArchivo,0,header=1)
verExcelOriginal.head()

MEDICO


,Linea,Representante,Codigo Medico,Codigo Inst,Tipo Inst,Tipo Pers,Paterno,Materno,Nombre,Calle,...,Cedula,Tel1,Tel2,Celular,Email,Rango Edad,Da Consulta en Hosp,Perfil,Fecha_Alta,Fecha_Mod
0,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{C2F2124D-BC32-4BA8-A9EE-A1AE0668CD46},{8B63AF92-F950-448A-893D-72DC8C9F6F98},CONSULTORIOS,MEDICO,ABREGO,CANTU,ISAIAS,PASEO DE LOS LEONES,...,919548,8183675421,NaN,NaN,Isaias.abrego@hotmail.com,40.0,NO,NaN,45299,NaN
1,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{828B138D-49E5-4B6B-8D0F-3CD28B775F92},{51FCB75E-9461-4A8F-841A-360431483F7E},CONSULTORIOS,MEDICO,AGUIRRE,LUIS,CECILIA DEL CARMEN,NOCHISTLAN,...,423876,8183330594,NaN,NaN,NaN,41.0,NO,NaN,45257,NaN
2,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{1DAA59E8-0C6C-4CE3-8F85-FF9569AC247D},{A7020994-2283-464E-B065-1AD08B1562AC},CONSULTORIOS,MEDICO,ALANIS,MARCOS,BERNARDO FRANCISCO,ECUADOR,...,3751264,8186873355,NaN,NaN,Doctor.bernardofco@gmail.com,50.0,NO,NaN,45257,NaN
3,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{65538A8B-1943-477B-AEC2-8CE88F958559},{912B89D3-74CE-435B-AD82-424CBA650202},CONSULTORIOS,MEDICO,ALCALA,FELIX,UBALDO,LA GAVIA,...,12152219,8183733978,NaN,NaN,drubaldoaf@gmail.com,52.0,NO,NaN,44992,NaN
4,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{EACAC27C-A908-491E-BC18-1B24A3D2F1E5},{8E75E97A-B2F0-4713-822A-BE31F0FF1087},CONSULTORIOS,MEDICO,ALCANTARA,RIVERA,MILITZA,PASEO DE LOS LEONES,...,10394903,8123211532,NaN,NaN,Dra.militza.alcantara@gmail.com,30.0,NO,NaN,45273,NaN


In [4]:
verExcelOriginal.columns

verExcel = verExcelOriginal.rename(columns={'Num Ext': 'Num_Ext'})

In [5]:
verExcel['Direccion']= verExcel["Calle"]
verExcel['Col']=verExcel["Colonia"].str[7:]
verExcel['Ciudad']=verExcel["Poblacion"]
verExcel["Pais"]="Mexico"

verExcel.head()

,Linea,Representante,Codigo Medico,Codigo Inst,Tipo Inst,Tipo Pers,Paterno,Materno,Nombre,Calle,...,Email,Rango Edad,Da Consulta en Hosp,Perfil,Fecha_Alta,Fecha_Mod,Direccion,Col,Ciudad,Pais
0,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{C2F2124D-BC32-4BA8-A9EE-A1AE0668CD46},{8B63AF92-F950-448A-893D-72DC8C9F6F98},CONSULTORIOS,MEDICO,ABREGO,CANTU,ISAIAS,PASEO DE LOS LEONES,...,Isaias.abrego@hotmail.com,40.0,NO,NaN,45299,NaN,PASEO DE LOS LEONES,LAS CUMBRES 2 SECTOR,MONTERREY,Mexico
1,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{828B138D-49E5-4B6B-8D0F-3CD28B775F92},{51FCB75E-9461-4A8F-841A-360431483F7E},CONSULTORIOS,MEDICO,AGUIRRE,LUIS,CECILIA DEL CARMEN,NOCHISTLAN,...,NaN,41.0,NO,NaN,45257,NaN,NOCHISTLAN,MIRTAS CENTRO,MONTERREY,Mexico
2,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{1DAA59E8-0C6C-4CE3-8F85-FF9569AC247D},{A7020994-2283-464E-B065-1AD08B1562AC},CONSULTORIOS,MEDICO,ALANIS,MARCOS,BERNARDO FRANCISCO,ECUADOR,...,Doctor.bernardofco@gmail.com,50.0,NO,NaN,45257,NaN,ECUADOR,BALCONES DE GALERIAS,MONTERREY,Mexico
3,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{65538A8B-1943-477B-AEC2-8CE88F958559},{912B89D3-74CE-435B-AD82-424CBA650202},CONSULTORIOS,MEDICO,ALCALA,FELIX,UBALDO,LA GAVIA,...,drubaldoaf@gmail.com,52.0,NO,NaN,44992,NaN,LA GAVIA,AZTLAN,MONTERREY,Mexico
4,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{EACAC27C-A908-491E-BC18-1B24A3D2F1E5},{8E75E97A-B2F0-4713-822A-BE31F0FF1087},CONSULTORIOS,MEDICO,ALCANTARA,RIVERA,MILITZA,PASEO DE LOS LEONES,...,Dra.militza.alcantara@gmail.com,30.0,NO,NaN,45273,NaN,PASEO DE LOS LEONES,LAS CUMBRES 2 SECTOR,MONTERREY,Mexico


In [6]:
def concatena_campos(row):
    return f"{row['Direccion']},{row['Col']},{row['Ciudad']},{row['Estado']},{row['Pais']}"

In [7]:
verExcel["noGeo"]= verExcel.apply(concatena_campos,axis=1)
verExcel.head()

verExcel.to_excel("archivodePrueba.xlsx")

In [8]:
verExcel_for_osm = verExcel.copy()
verExcel_for_mapbox = verExcel.copy()
verExcel_for_arcgis = verExcel.copy()

geocodificacion completa con la API de OSM

In [20]:
geolocator =Nominatim(user_agent="jorgemc",timeout=5)
geocode = partial(geolocator.geocode,language = "es")

def tidygeocode(row):
    location = geolocator.geocode(row['noGeo'])
    if location:
        return pd.Series({'latitude':location.latitude,"longitude":location.longitude})
    else:
        return pd.Series({'latitude':None,'longitude':None})
    



In [ ]:
verExcel_for_osm[['latitude','longitude']] = verExcel_for_osm.apply(tidygeocode,axis=1)

verExcel_for_osm

Geolocalizando con ArcGis

In [9]:
api_arcgis = "AAPKe017698084d742049005e493d4bce9ddY45j51f68jQMWf7cA43OuO3zs160_aYigr1xskjzOzm23Y8n3ijvKA5DlYurkuOq"
geolocator = ArcGIS(auth_domain=api_arcgis)

def tidygeocode(row):
    location = geolocator.geocode(row['noGeo'])
    if location:
        return pd.Series({'latitude':location.latitude,"longitude":location.longitude})
    else:
        return pd.Series({'latitude':None,'longitude':None})
    

In [10]:
verExcel_for_arcgis[['latitude','longitude']] = verExcel_for_osm.apply(tidygeocode,axis=1)

verExcel_for_arcgis

,Linea,Representante,Codigo Medico,Codigo Inst,Tipo Inst,Tipo Pers,Paterno,Materno,Nombre,Calle,...,Rango Edad,Da Consulta en Hosp,Perfil,Fecha_Alta,Fecha_Mod,Direccion,Col,Ciudad,Pais,noGeo
0,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{C2F2124D-BC32-4BA8-A9EE-A1AE0668CD46},{8B63AF92-F950-448A-893D-72DC8C9F6F98},CONSULTORIOS,MEDICO,ABREGO,CANTU,ISAIAS,PASEO DE LOS LEONES,...,40.0,NO,NaN,45299,NaN,PASEO DE LOS LEONES,LAS CUMBRES 2 SECTOR,MONTERREY,Mexico,"PASEO DE LOS LEONES, LAS CUMBRES 2 SECTOR,MONT..."
1,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{828B138D-49E5-4B6B-8D0F-3CD28B775F92},{51FCB75E-9461-4A8F-841A-360431483F7E},CONSULTORIOS,MEDICO,AGUIRRE,LUIS,CECILIA DEL CARMEN,NOCHISTLAN,...,41.0,NO,NaN,45257,NaN,NOCHISTLAN,MIRTAS CENTRO,MONTERREY,Mexico,"NOCHISTLAN, MIRTAS CENTRO,MONTERREY,NUEVO LEON..."
2,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{1DAA59E8-0C6C-4CE3-8F85-FF9569AC247D},{A7020994-2283-464E-B065-1AD08B1562AC},CONSULTORIOS,MEDICO,ALANIS,MARCOS,BERNARDO FRANCISCO,ECUADOR,...,50.0,NO,NaN,45257,NaN,ECUADOR,BALCONES DE GALERIAS,MONTERREY,Mexico,"ECUADOR, BALCONES DE GALERIAS,MONTERREY,NUEVO ..."
3,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{65538A8B-1943-477B-AEC2-8CE88F958559},{912B89D3-74CE-435B-AD82-424CBA650202},CONSULTORIOS,MEDICO,ALCALA,FELIX,UBALDO,LA GAVIA,...,52.0,NO,NaN,44992,NaN,LA GAVIA,AZTLAN,MONTERREY,Mexico,"LA GAVIA, AZTLAN,MONTERREY,NUEVO LEON,Mexico"
4,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{EACAC27C-A908-491E-BC18-1B24A3D2F1E5},{8E75E97A-B2F0-4713-822A-BE31F0FF1087},CONSULTORIOS,MEDICO,ALCANTARA,RIVERA,MILITZA,PASEO DE LOS LEONES,...,30.0,NO,NaN,45273,NaN,PASEO DE LOS LEONES,LAS CUMBRES 2 SECTOR,MONTERREY,Mexico,"PASEO DE LOS LEONES, LAS CUMBRES 2 SECTOR,MONT..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{6842D0D4-3478-457C-AA8B-0B4C5D3A1D38},{CD738C67-C2BC-4E1B-A697-8445347ADB82},CONSULTORIOS,MEDICO,VILLARREAL,RODRIGUEZ,RICARDO JESUS,FAROLILLO,...,40.0,NO,NaN,45264,NaN,FAROLILLO,FOMERREY 109,MONTERREY,Mexico,"FAROLILLO, FOMERREY 109,MONTERREY,NUEVO LEON,M..."
216,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{B5754357-05A9-4BA0-AF50-56E0547520D0},{C8E9FD4F-A947-494E-83E9-75FE0DE19A08},CONSULTORIOS,MEDICO,ZAVALA,ELIZONDO,ALEJANDRO,ECUADOR,...,45.0,NO,NaN,45264,NaN,ECUADOR,BALCONES DE GALERIAS,MONTERREY,Mexico,"ECUADOR, BALCONES DE GALERIAS,MONTERREY,NUEVO ..."
217,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{AC7EE1D1-AACA-4263-89F7-0278F433EACF},{D5A82513-24E4-450C-86EE-0C64CC5AF7AF},CONSULTORIOS,MEDICO,ZERTUCHE,GARZA,HECTOR MARIO,PASEO DE LOS LEONES,...,35.0,NO,NaN,45401,NaN,PASEO DE LOS LEONES,LAS CUMBRES 2 SECTOR,MONTERREY,Mexico,"PASEO DE LOS LEONES, LAS CUMBRES 2 SECTOR,MONT..."
218,RAYERE,P1707 - FRIAS TORRES HUMBERTO,{D9872BDD-836A-4E2F-BFC2-B83844793828},{B4E12260-BB65-4945-BEDA-4D7BF06AA75C},HOSPITALES,MEDICO,ZORRILLA,FIERRO,CECILIA,PASEO DE LOS LEONES,...,34.0,SI,NaN,44971,45264.0,PASEO DE LOS LEONES,CUMBRES 1ER. SECTOR,MONTERREY,Mexico,"PASEO DE LOS LEONES, CUMBRES 1ER. SECTOR,MONTE..."


Se creo un Dataframe con geometria y con una Referencia de sistema de Coordenadas.   en este caso tenemos un sistema de coordenasas ITRF 2008 y Geometria de Puntos. 

In [11]:
geo_rawData = gpd.GeoDataFrame(data=verExcel_for_arcgis,geometry=gpd.points_from_xy(verExcel_for_arcgis.longitude,verExcel_for_arcgis.latitude),crs=6363)

Abrir mapa para mostrar datos.

In [ ]:
geo_rawData.explore()

Convertir el archivo a un GeoJson o Un Shapefile

In [13]:
geo_rawData.to_file('MediccosP1707.shp')


verExcel_for_arcgis.to_excel("MedicosP1707.xlsx")
#geo_rawData.to_file('Hospitales_Herrera.geojson',driver="GeoJSON")

C:\Users\Invmerp01\AppData\Local\Temp\ipykernel_24940\4037618448.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  geo_rawData.to_file('MediccosP1707.shp')
c:\Users\Invmerp01\AppData\Local\Programs\Python\Python310\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Representante' to 'Representa'
  ogr_write(
c:\Users\Invmerp01\AppData\Local\Programs\Python\Python310\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Codigo Medico' to 'Codigo Med'
  ogr_write(
c:\Users\Invmerp01\AppData\Local\Programs\Python\Python310\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Codigo Inst' to 'Codigo Ins'
  ogr_write(
c:\Users\Invmerp01\AppData\Local\Programs\Python\Python310\lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Especialidad' to 'Especialid'
  ogr_write(
c:\Users\Invmerp01\AppDat